In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
submission_pattern = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
submission_pattern.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [5]:
X = train_data.drop(['Name','Embarked','Cabin','Survived','Ticket'],axis =1)
X.isnull().sum(axis=0)

PassengerId      0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

In [6]:
X = X.drop(['Age'],axis=1)

In [7]:
X_train = pd.get_dummies(X)
X_train.head()

,PassengerId,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male
0,1,3,1,0,7.2500,False,True
1,2,1,1,0,71.2833,True,False
2,3,3,0,0,7.9250,True,False
3,4,1,1,0,53.1000,True,False
4,5,3,0,0,8.0500,False,True


In [8]:
dropped_features = ['Name','Embarked','Cabin','Ticket','Age']
y_train = train_data['Survived']
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [9]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators =100)

In [10]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [11]:
X_test = test_data.drop(dropped_features,axis=1)
X_test = X_test.fillna(X_test['Fare'].mean())
X_test = pd.get_dummies(X_test)
X_test.tail()

,PassengerId,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male
413,1305,3,0,0,8.0500,False,True
414,1306,1,0,0,108.9000,True,False
415,1307,3,0,0,7.2500,False,True
416,1308,3,0,0,8.0500,False,True
417,1309,3,1,1,22.3583,False,True


In [12]:
preds = model.predict(X_test)
preds.shape

(418,)

In [13]:
ids = X_test['PassengerId'].values

In [14]:
ans = pd.DataFrame({'PassengerId': ids, 'Survived': preds})
ans.head(20)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


In [15]:
ans.to_csv('Submission.csv',index=False)